# Digital Library

In [1]:
import subprocess
import os
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import UnstructuredPDFLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
import chromadb

In [33]:
docsPath='/nfs/turbo/umms-indikar/shared/projects/RAG/papers/DigitalLibrary-9-June-2024/'
dbName='DigitalLibrary'
dbPath='/nfs/turbo/umms-indikar/shared/projects/RAG/databases/DigitalLibrary-10-June-2024/'
HuggingFaceEmbeddingsModel = 'BAAI/bge-base-en-v1.5'
chunk_size=[1000]
chunck_overlap=[50]
v=True

In [3]:
local = os.getcwd()  ## Get local dir
os.chdir(local)      ## shift the work dir to local dir

print('\nWork Directory: {}'.format(local)) if v else None

#%% Phase 1 - Load DB
embeddings_model = HuggingFaceEmbeddings(model_name=HuggingFaceEmbeddingsModel)

print('\nDocuments loading from:', docsPath) if v else None

text_loader_kwargs={'autodetect_encoding': True}


Work Directory: /home/jpic/RAG-DEV/experimental


/home/jpic/.local/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/home/jpic/.local/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(



Documents loading from: /nfs/turbo/umms-indikar/shared/projects/RAG/papers/DigitalLibrary-9-June-2024/


In [4]:
loader = DirectoryLoader(docsPath,
                         glob="**/*.pdf",
                         loader_cls=PyPDFLoader, 
                         #loader_kwargs=text_loader_kwargs,
                         show_progress=True,
                         )
docs_data = loader.load()

  0%|          | 2/520 [00:01<05:37,  1.53it/s]Ignoring wrong pointing object 15 0 (offset 0)
Ignoring wrong pointing object 39 0 (offset 0)
Ignoring wrong pointing object 45 0 (offset 0)
 31%|███       | 161/520 [09:58<20:23,  3.41s/it]  Ignoring wrong pointing object 11 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
Ignoring wrong pointing object 55 0 (offset 0)
Ignoring wrong pointing object 98 0 (offset 0)
Ignoring wrong pointing object 134 0 (offset 0)
Ignoring wrong pointing object 147 0 (offset 0)
Ignoring wrong pointing object 149 0 (offset 0)
Ignoring wrong pointing object 164 0 (offset 0)
Ignoring wrong pointing object 186 0 (offset 0)
Ignoring wrong pointing object 188 0 (offset 0)
Ignoring wrong pointing object 197 0 (offset 0)
 86%|████████▌ | 445/520 [21:58<02:10,  1.74s/it]  Ignoring wrong pointing object 2 65536 (offset 0)
Ignoring wrong pointing object 14 65536 (offset 0)
Ignoring wrong pointing object 35 65536 (offset 0)
Ignoring wrong pointing object 55 

In [ ]:
chunk_size = [1000] #Chunk size 
chunk_overlap = [50] #Chunk overlap
for i in range(len(chunk_size)):
    for j in range(len(chunk_overlap)):
        text_splitter = RecursiveCharacterTextSplitter(chunk_size = chunk_size[i],
                                                        chunk_overlap = chunk_overlap[j],
                                                        separators=[" ", ",", "\n", ". "])
        
        data_splits = text_splitter.split_documents(docs_data)
        print('Split Documents: ' + str(len(data_splits)))

        p2_2 = subprocess.run('mkdir  %s/*'%(dbPath+dbName), shell=True)
        _client_settings = chromadb.PersistentClient(path=(dbPath+dbName))

        print('Beginning Chrom Call')
        vectordb = Chroma.from_documents(documents           = data_splits,
                                         embedding           = embeddings_model,
                                         client              = _client_settings,
                                         collection_name     = dbName,
                                         collection_metadata = {"hnsw:space": "cosine"})

        print('Completed Chroma Database: ', dbName) if v else None

Split Documents: 62202


mkdir: cannot create directory ‘/nfs/turbo/umms-indikar/shared/projects/RAG/databases/DigitalLibrary-10-June-2024/DigitalLibrary/*’: No such file or directory


Beginning Chrom Call


# Load the Database

In [2]:
persist_directory = '/nfs/turbo/umms-indikar/shared/projects/RAG/databases/DigitalLibrary-10-June-2024/' # DigitalLibraryXXX' # '/nfs/turbo/umms-indikar/shared/projects/RAG/databases/DigitalLibrary-10-June-2024/DigitalLibrary/'
embeddings_model = HuggingFaceEmbeddings(model_name='BAAI/bge-base-en-v1.5')        # Embedding model
db_name = "DigitalLibrary" # "DB_cosine_cSize_%d_cOver_%d" % (700, 200)
_client_settings = chromadb.PersistentClient(path=(persist_directory + db_name))
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embeddings_model, client=_client_settings, collection_name=db_name)

/home/jpic/.local/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/home/jpic/.local/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
docs = vectordb.similarity_search_with_relevance_scores('Remark on the differences between direct and indirect cell reprogramming', k=4)
docs

In [43]:
persist_directory

'/nfs/turbo/umms-indikar/shared/projects/RAG/databases/DigitalLibrary-10-June-2024/DigitalLibraryDigitalLibrary'

In [44]:
db_name

'DB_cosine_cSize_700_cOver_200'

In [4]:
len(vectordb.get()['documents'])

KeyboardInterrupt: 

In [5]:
len(vectordb.get()['ids'])

62202

In [12]:
docs = vectordb.similarity_search_with_relevance_scores('Remark on the differences between direct and indirect cell reprogramming', k=4)
docs

[(Document(page_content='without transitioning through an intermediate pluripotent \nor multi  potent state\n3, has been described as ‘direct  \nreprogramming’ , also known as ‘transdifferentiation’ .\nIn addition to increasing our understanding of cell \nfate specification and plasticity, direct reprogramming \nholds promises for regenerative medicine. Compared to induced pluripotent stem cell (iPSC) reprogramming \n(not the focus of this review, reviewed in \nrefs4–7), direct \nreprogramming is a faster and more efficient process and has unique advantages for tissue repair \n(fig.\xa0 1). Whereas \nthe use of iPSCs6 requires the isolation of somatic cells \nand their reprogramming to a pluripotent state fol-lowed by their differentiation into a different lineage,  in principle, direct reprogramming enables the con-\nversion of cells in\xa0situ (in the desired tissue) without \ntransitioning through an intermediate pluripotent state \nand without the need for ex vivo cell expansion an

In [11]:
docs

[(Document(page_content='58 A Appendix: High-Accuracy Approximation Methods for General Binary-State. . .\n“spin up” versus “spin down” magnetic dipoles in an Ising spin model, “inactive”\nversus “active” nodes in a social system, and so on. We presented many examples of\nbinary-state dynamics in Chapter 3. See, for example, our discussions of percolation\nmodels in Sec. 3.1, biological contagions in Sec. 3.2, and social contagions in\nSec.3.3.\nIn the present section, we will consider asynchronous updating and local rules\nfor node updates. In other words, the transition rates depend only on the state of\na node and on the states of its immediate neighbors in a network. Consequently,\nan updating node that is susceptible becomes infected with a probability of Fk;mdt,\nwhere kis the node’s degree and mis the number of its neighbors that are infected.\nWe refer to the function Fk;mas the infection rate . Similarly, an updating node that is\ninfected becomes susceptible with a probabilit

In [42]:
help(Chroma)

Help on class Chroma in module langchain_community.vectorstores.chroma:

class Chroma(langchain_core.vectorstores.VectorStore)
 |  Chroma(collection_name: 'str' = 'langchain', embedding_function: 'Optional[Embeddings]' = None, persist_directory: 'Optional[str]' = None, client_settings: 'Optional[chromadb.config.Settings]' = None, collection_metadata: 'Optional[Dict]' = None, client: 'Optional[chromadb.Client]' = None, relevance_score_fn: 'Optional[Callable[[float], float]]' = None) -> 'None'
 |  
 |  `ChromaDB` vector store.
 |  
 |  To use, you should have the ``chromadb`` python package installed.
 |  
 |  Example:
 |      .. code-block:: python
 |  
 |              from langchain_community.vectorstores import Chroma
 |              from langchain_community.embeddings.openai import OpenAIEmbeddings
 |  
 |              embeddings = OpenAIEmbeddings()
 |              vectorstore = Chroma("langchain_store", embeddings)
 |  
 |  Method resolution order:
 |      Chroma
 |      langchain_

# Test

In [28]:
from BRAD import brad
vectordb, _ = brad.load_literature_db(persist_directory='/nfs/turbo/umms-indikar/shared/projects/RAG/databases/DigitalLibrary-10-June-2024/DigitalLibraryDigitalLibrary/')

Mon Jun 10 09:37:43 2024 INFO Use pytorch device_name: cpu
Mon Jun 10 09:37:43 2024 INFO Load pretrained SentenceTransformer: BAAI/bge-base-en-v1.5
Mon Jun 10 09:37:43 2024 INFO Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


In [36]:
len(vectordb.get()['documents'])

0

In [30]:
print(vectordb.get())

{'ids': [], 'embeddings': None, 'metadatas': [], 'documents': [], 'uris': None, 'data': None}


In [19]:
import umap
import random
import pandas as pd
import seaborn as sns
import subprocess
import matplotlib.pyplot as plt
import numpy as np
import chromadb
from langchain_chroma import Chroma
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from bert_score import BERTScorer
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_text_splitters import CharacterTextSplitter
from langchain.docstore.document import Document
from langchain_chroma import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import UnstructuredPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [20]:
from BRAD import brad

[nltk_data] Downloading package words to /home/jpic/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jpic/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/jpic/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [24]:
brad.chat(persist_directory='/nfs/turbo/umms-indikar/shared/projects/RAG/databases/DigitalLibrary-9-June-2024/DigitalLibraryDigitalLibrary/')

Welcome to RAG! The chat log from this conversation will be saved to /home/jpic/BRAD/2024-06-10-02:38:15.066406.json. How can I help?

Would you like to use a database with BRAD [Y/N]?


 Y


Mon Jun 10 02:38:20 2024 INFO Use pytorch device_name: cpu
Mon Jun 10 02:38:20 2024 INFO Load pretrained SentenceTransformer: BAAI/bge-base-en-v1.5
/home/jpic/.local/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Mon Jun 10 02:38:20 2024 INFO Collection DB_cosine_cSize_700_cOver_200 is not created.
Mon Jun 10 02:38:20 2024 INFO local


Input >>  /force RAG what can you tell me about bifurcation theory?


Mon Jun 10 02:38:38 2024 INFO RAG


...

Input >>  /force RAG what did smale contribute to dynamical systems?


Mon Jun 10 02:39:37 2024 INFO RAG


RAG >> 2: 

> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------

Human: /force RAG what did smale contribute to dynamical systems?

> Finished chain.

> Finished chain.
 Smale was a prominent mathematician who made significant contributions to the field of dynamical systems. Here are a few of his major accomplishments:

1. He proved the Poincaré conjecture in higher dimensions for certain cases using his h-cobordism theorem. This work contributed to the development of the modern theory of smooth manifolds and topology.
2. Smale introduced the concept of horseshoes and hyperbolic sets in dynamics. These objects are now fundamental in understanding chaotic behaviors in systems.
3. He proved the existence of a horseshoe map in the tent map and provided a t

Input >>  q


Thanks for chatting today! I hope to talk soon, and don't forget that a record of this conversation is available at: /home/jpic/BRAD/2024-06-10-02:38:15.066406.json


In [21]:
help(brad.chat)

Help on function chat in module BRAD.brad:

chat(model_path='/nfs/turbo/umms-indikar/shared/projects/RAG/models/llama-2-7b-chat.Q8_0.gguf', persist_directory='/nfs/turbo/umms-indikar/shared/projects/RAG/databases/Transcription-Factors-5-10-2024/', llm=None, ragvectordb=None, embeddings_model=None)
    Initializes and runs the RAG chatbot, allowing interaction with various models and databases, and logs the conversation.
    
    :param model_path: The path to the Llama model file, defaults to '/nfs/turbo/umms-indikar/shared/projects/RAG/models/llama-2-7b-chat.Q8_0.gguf'.
    :type model_path: str, optional
    :param persist_directory: The directory where the literature database is stored, defaults to "/nfs/turbo/umms-indikar/shared/projects/RAG/databases/Transcription-Factors-5-10-2024/".
    :type persist_directory: str, optional
    :param llm: The language model to be used. If None, it will be loaded within the function.
    :type llm: PreTrainedModel, optional
    :param ragvector